# Workshop 0: Hello, AI!

Welcome! In this workshop, you'll have a conversation with an AI — and discover a few surprising things about how it actually works.

**You don't need to write any code.** Everything is pre-written. You just press **Shift+Enter** to run each cell and observe what happens.

Let's start by setting things up.

In [1]:
# Just run this cell! Press Shift+Enter
# This connects us to the AI service

import os
from openai import OpenAI

API_KEY = os.getenv("API_KEY") or os.getenv("OPENAI_API_KEY") or os.getenv("ANTHROPIC_API_KEY") or os.getenv("GEMINI_API_KEY")

if API_KEY and API_KEY.startswith("sk-ant-"):
    BASE_URL = "https://api.anthropic.com/v1/"
    MODEL = "claude-sonnet-4-20250514"
elif API_KEY and API_KEY.startswith("sk-"):
    BASE_URL = "https://api.openai.com/v1"
    MODEL = "gpt-4o"
else:
    BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
    MODEL = "gemini-2.0-flash"

BASE_URL = os.getenv("OPENAI_BASE_URL", BASE_URL)
MODEL = os.getenv("MODEL", MODEL)

client = OpenAI(api_key=API_KEY, base_url=BASE_URL)

print(f"Connected! Using model: {MODEL}")

Connected! Using model: gpt-4o


---

## Section 1: Talking to an AI

Talking to an AI is a lot like **sending a text message**. You type something, send it, and get a reply back.

Behind the scenes, your message goes to an AI model (like ChatGPT, Claude, or Gemini), and it sends back a response. That's it. No magic — just a request and a response.

Let's send our first message.

In [ ]:
# Just run this cell! Press Shift+Enter
# We're sending a simple message to the AI

response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "user", "content": "What are three fun facts about the Moon?"}
    ]
)

print(response.choices[0].message.content)

**That's it!** You just sent a message to an AI and got a reply.

Every AI product you've ever used — ChatGPT, Copilot, Gemini — is doing exactly this under the hood: sending messages and receiving responses.

Now let's discover something surprising about how the AI's memory works...

---

## Section 2: AI Has No Memory

Imagine you're texting someone who has **complete amnesia after every message**. Each time you text them, they have zero recollection of anything you've said before. It's like talking to a goldfish with a PhD.

Sounds weird, right? Let's see it in action.

**Step 1:** We'll introduce ourselves to the AI.

In [ ]:
# Just run this cell! Press Shift+Enter
# Let's tell the AI our name

response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "user", "content": "Hi there! My name is Ralph. Nice to meet you!"}
    ]
)

print(response.choices[0].message.content)

Great, the AI greeted us by name. It clearly knows who we are.

**Step 2:** Now let's ask it what our name is — in a *separate* message.

In [ ]:
# Just run this cell! Press Shift+Enter
# Let's ask the AI if it remembers our name

response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "user", "content": "What is my name?"}
    ]
)

print(response.choices[0].message.content)

### Wait... it forgot?!

**Yes!** The AI has absolutely no memory between calls. Each time you send a message, it's a brand new conversation. The AI has never seen you before.

This is one of the most important things to understand about AI:

> **Every single request to an AI starts from scratch.** It doesn't remember anything unless *you* remind it.

Think about what this means for products like ChatGPT. When you have a long conversation, the AI doesn't actually *remember* any of it. So how does it seem like it does?

Let's find out.

---

## Section 3: The Chat History Trick

Here's the trick: **we keep a notebook of the entire conversation and show it to the AI every single time.**

Imagine you're texting that amnesiac friend again. But this time, before every message, you copy-paste the *entire conversation so far* at the top. Now your friend can read the history and respond as if they remember everything.

That's exactly what ChatGPT does behind the scenes. Let's try it.

In [ ]:
# Just run this cell! Press Shift+Enter
# This time we send the FULL conversation history

response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "user", "content": "Hi there! My name is Ralph. Nice to meet you!"},
        {"role": "assistant", "content": "Hello Ralph! Nice to meet you too! How can I help you today?"},
        {"role": "user", "content": "What is my name?"},
    ]
)

print(response.choices[0].message.content)

### It remembers!

But it doesn't *really* remember. We just **showed it the full conversation** and it read through it to give us a consistent answer.

This is exactly how every AI chatbot works:
1. You send a message
2. The app stores the message in a list
3. Next time, the app sends the **entire list** back to the AI
4. The AI reads the whole thing and responds

### But there's a catch...

This "notebook" has a **limited number of pages**. You can't copy-paste an infinite conversation. At some point, you run out of space.

This limit is called the **context window** — it's the maximum amount of text the AI can read at once. Think of it like a desk: you can only spread out so many papers before things start falling off the edges.

| Model | Context Window | Rough Equivalent |
|-------|---------------|-------------------|
| GPT-4o | 128K tokens | ~200 pages of text |
| Claude Sonnet | 200K tokens | ~300 pages of text |
| Gemini Flash | 1M tokens | ~1,500 pages of text |

**PM Insight:** When you're evaluating an AI product, ask: *"How does it handle conversations that exceed the context window?"* The answer reveals a lot about how thoughtfully the product was built.

---

## Section 4: The System Prompt — Giving AI a Personality

So far we've been sending messages as the `user`. But there's a special type of message called the **system prompt**. It's like giving the AI a job description before the conversation starts.

Think of it this way:
- The **system prompt** is like a director's brief to an actor: *"You're playing a pirate captain in this scene."*
- The **user messages** are the lines from the other actors.

The AI will stay in character for the whole conversation based on the system prompt.

Let's see it in action.

In [ ]:
# Just run this cell! Press Shift+Enter
# We give the AI a "personality" using a system prompt

response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a pirate captain. Respond to everything in pirate speak. Use plenty of 'arrr', 'matey', and 'shiver me timbers'. Keep responses short and fun."},
        {"role": "user", "content": "What's the best programming language?"},
    ]
)

print(response.choices[0].message.content)

### Your turn! Try changing the personality.

In the cell below, **edit the system prompt** to make the AI talk like someone else. Some ideas:
- A Bollywood villain delivering a monologue
- An over-excited cricket commentator
- A Shakespearean poet
- A motivational gym coach
- Your favourite movie character

Just change the text inside the quotes next to `"content":` in the system message, then press **Shift+Enter**.

In [ ]:
# Edit the system prompt below, then press Shift+Enter to run!

response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a dramatic Bollywood villain. Every response must include a dramatic pause (...), a reference to destiny, and end with an evil laugh. Keep it fun and over-the-top."},
        {"role": "user", "content": "Should I learn about AI?"},
    ]
)

print(response.choices[0].message.content)

**PM Insight:** The system prompt is incredibly powerful. In real products, this is where you define:
- The AI's tone and personality (friendly? formal? fun?)
- What the AI should and shouldn't talk about
- How it should format responses
- Safety guardrails and boundaries

When you're building an AI product, the system prompt is essentially your **product requirements document for the AI's behavior**.

---

## Section 5: Recap — What You Just Learned

You've just discovered the core building blocks that power every AI product. Let's recap:

### Key Takeaways

**1. Talking to AI = sending a message and getting a response**
> Under the hood, every AI interaction is just an API call. The fancy chat interfaces you see are just wrappers around this simple exchange.

**2. AI is stateless — it has no memory**
> Every request starts from zero. The AI doesn't remember past conversations. *You* (the product) must manage the conversation history.

**3. The "memory" trick = sending the full chat history every time**
> Products like ChatGPT make AI *seem* like it remembers by replaying the entire conversation with each request.

**4. Context window = limited memory budget**
> There's a hard limit on how much text the AI can process at once. Longer conversations and bigger documents mean tougher design decisions.

**5. System prompt = product requirements for AI behavior**
> This is where you shape the AI's personality, boundaries, and behavior. It's one of the most important levers in AI product design.

---

**These five concepts are the foundation of every AI agent.** In the next workshops, we'll build on them to create AI that can use tools, make decisions, and work autonomously.